# MQTT2OPC  
OPC2MQTT是从OPC获取数据以后通过MQTT发送,MQTT2OPC是接收MQTT的数据写入OPC,相应的各自的CSV配置为opc2mqtt.csv和mqtt2opc文件。

In [1]:
# app.py

import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# create a file handler
handler = logging.FileHandler('mqtt2opc.log',encoding='utf-8')
handler.setLevel(logging.INFO)

# create a logging format
formatter = logging.Formatter('%(asctime)s,%(name)s,%(levelname)s,%(message)s')
handler.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(handler)

### STEP2:连接MQTT服务器

In [2]:
import paho.mqtt.client as mqtt #import the client1
broker_address="broker.emqx.io"
#broker_address="iot.eclipse.org" #use external broker
mqttc = mqtt.Client() #create new instance
mqttc.connect(broker_address) #connect to broker

0

### STEP3: 加载CSV配置文件

In [3]:
import csv

In [4]:
with open('mqtt2opc.csv', newline='') as f:
    reader = csv.reader(f)
    header = next(reader)
    list_reader = list(reader)
    print(list_reader)

[['srm/mqtt2opc/INT_NUM', 'ns=3;s="OPC2MQTT"."INT_NUM"'], ['srm/mqtt2opc/B_RUN', 'ns=3;s="OPC2MQTT"."B_RUN"']]


In [5]:
list_topic = [i[0] for i in list_reader]

In [6]:
list_topic

['srm/mqtt2opc/INT_NUM', 'srm/mqtt2opc/B_RUN']

In [7]:
list_nodeid = [i[1] for i in list_reader]

In [8]:
list_nodeid

['ns=3;s="OPC2MQTT"."INT_NUM"', 'ns=3;s="OPC2MQTT"."B_RUN"']

In [9]:
dict_topic_nodeid = dict(zip(list_topic, list_nodeid))

In [10]:
dict_topic_nodeid.get('srm/mqtt2opc/INT_NUM')

'ns=3;s="OPC2MQTT"."INT_NUM"'

### STEP1: 连接OPC服务器

In [11]:
""" Step 1 """
from opcua import Client as OPCUAClient

In [12]:
from opcua import ua

In [13]:
uri = "opc.tcp://192.168.10.1:4840"

In [14]:
""" Step 2 """
opcc = OPCUAClient(uri)

In [15]:
try:
    opcc.connect() # connect to the data source
except Exception as e:
    logging.exception(e)

Requested session timeout to be 3600000ms, got 30000ms instead


### STEP4: 读取MQTT中的数据写入OPC

In [16]:
import json
def subscribe(client,list_topic, list_nodeid):

    dict_topic_nodeid = dict(zip(list_topic, list_nodeid))
    
    def on_message(client, userdata, msg):
#         data = msg.payload
#         receive=data.decode("utf-8")
#         m_decode = json.loads(receive)
#         #print(m_decode)
#         #print (m_decode['Server_name'])
#         print ("Message received: "  + str(m_decode))
        
        print(f"Received '{msg.payload.decode()}' from topic'{msg.topic}'")
        value = json.loads(msg.payload.decode('utf-8'))
        nodeid = dict_topic_nodeid.get(msg.topic)

        node = opcc.get_node(nodeid)
        node.set_value(ua.DataValue(ua.Variant(value, node.get_data_type_as_variant_type())))
        
    
    
    client.subscribe([(i,0) for i in list_topic])
    client.on_message = on_message

In [ ]:
subscribe(mqttc, list_topic, list_nodeid)
mqttc.loop_forever()

Received '1' from topic'srm/mqtt2opc/INT_NUM'
